In [9]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
# import deepchem as dc
import transformers
os.environ["KERAS_BACKEND"] = "tensorflow"


In [10]:
# %pip install cv2
# !pip show cv2
DATA_PATH = 'CompiledDataset/relevant_data.csv'
SMILES_PATH = 'CompiledDataset/data.csv'

import sys
from scipy import spatial
!{sys.executable} -m pip install opencv-python
import cv2
import pandas as pd
# from Model import GetData, DATA_PATH

#Retrieve dataframe from stored csv file
def GetData(file):
    """
    Retrieve data from file, return pandas dataframe
    """
    d = pd.read_csv(file)
    Data = d['Images']
    Data = pd.concat([Data, d[[str(i) for i in range(128)]]], axis="columns")
    Data['SMILES'] = pd.read_csv(SMILES_PATH)['SMILES']
    return Data

def _getEmbedding(index, df):
    """
    Return an embedding from dataframe (consisting of only the embeddings)
    """
    return df.iloc[index].to_numpy()

def _get_closest_embedding(vector, embeds):
    #euclidean = lambda x, y: 
    #distances = {embed:spatial.distance.cosine(vector, embed) for embed in embeds}
    #sorted_dists = sorted(distances.items(), key=lambda x:x[1])
    distances = [(embed, spatial.distance.cosine(vector, embed)) for embed in embeds]
    sorted_dists = sorted(distances, key=lambda x:x[1])
    return sorted_dists[0]


/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [11]:
data = GetData(DATA_PATH)

In [12]:
data

,Images,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,SMILES
0,ImageData/uh0.png,0.008918,0.007185,0.003843,-0.004025,-0.007159,-0.004339,-0.002224,0.004571,-0.006694,-0.007999,-0.004291,0.011735,-0.006531,0.001966,0.006885,0.006013,0.000662,0.005516,-0.006479,0.004861,-0.000184,-0.005223,-0.001973,-0.007196,-0.000755,-0.000822,0.009544,-0.005357,-0.008139,-0.000345,-0.007473,-0.006915,-0.001352,-0.004765,-0.006354,0.000918,-0.001717,0.003844,0.005029,...,-0.000821,0.005105,-0.004275,-0.000799,0.008673,0.000274,0.008107,-0.002080,-0.000028,0.005188,-0.005440,-0.002983,-0.005625,0.003339,0.006075,0.008605,0.004375,0.008291,-0.007533,-0.004409,-0.004594,-0.002282,-0.004143,-0.001228,-0.001946,0.003032,0.004576,-0.006307,0.007851,0.004956,-0.005933,0.002324,-0.000274,0.001278,0.006330,-0.000406,0.005050,0.003521,0.005719,C[NH+](CC1=C(C=CC=C1Cl)F)CN2C(=O)CC3(C2=O)CCCCC3
1,ImageData/uh1.png,0.005314,0.007546,-0.005600,0.001770,0.004105,-0.003233,0.001038,0.001464,-0.009206,-0.000888,0.001869,0.004732,-0.004145,-0.006327,-0.000093,0.002243,0.000372,-0.004709,-0.002027,0.004408,-0.010198,-0.001391,0.002642,-0.004515,-0.006944,-0.002700,-0.004633,-0.007544,-0.000347,-0.005007,-0.002446,-0.008256,0.010512,0.001191,-0.006567,-0.002315,-0.002737,0.000054,0.007359,...,-0.004764,-0.001224,0.004128,0.001924,-0.003739,0.004649,0.003892,0.007169,0.004983,0.014873,-0.010631,0.004798,0.000373,-0.001231,-0.007353,0.009226,-0.008970,0.008171,0.003407,0.008012,0.005798,-0.003944,-0.002898,0.000217,-0.001001,-0.005929,-0.005656,-0.002551,0.009402,-0.000326,0.001146,0.002944,0.003084,0.001922,0.002732,-0.005300,-0.003507,0.004510,0.000134,CC1=C(C=C(C=C1)C(=O)NC2=C(C=C(C=C2)I)C3=NC4=CC...
2,ImageData/uh2.png,0.010540,-0.001101,-0.010949,0.010823,0.006059,-0.007081,0.004326,-0.004051,-0.007896,-0.000953,-0.006207,0.001808,-0.002122,0.000971,0.005603,0.002528,0.010361,-0.002384,-0.004876,-0.004450,-0.008519,-0.005696,0.003453,-0.008112,-0.001826,0.003239,0.006671,-0.006834,-0.004203,-0.000424,-0.000932,0.002791,0.010567,-0.002385,-0.005497,0.010194,0.001709,-0.000711,-0.000535,...,-0.006438,-0.004001,-0.004628,0.005718,-0.001289,-0.007314,0.009702,0.007306,-0.004676,0.009206,-0.011803,-0.004486,-0.004700,-0.002715,-0.005141,-0.005349,-0.006808,0.005513,-0.006714,0.003355,-0.003378,-0.001550,0.002835,0.001756,0.003227,0.005187,-0.000590,-0.014790,-0.005416,0.004891,0.001325,-0.007659,-0.009448,-0.005587,-0.004743,0.007540,-0.004380,-0.006842,0.008730,COC1=CC=C(C=C1)CCN2C(=O)CC(SC2=NC3=CC=C(C=C3)O...
3,ImageData/uh3.png,0.009642,-0.005712,-0.007004,-0.001866,-0.005760,-0.001075,0.004211,-0.006042,-0.010296,-0.001695,0.004449,0.013002,-0.004568,0.006260,-0.001988,0.000253,0.000721,0.000360,-0.003678,0.008282,-0.010458,-0.010255,-0.000651,0.005504,-0.001483,-0.000679,0.010722,-0.003518,-0.010435,0.001080,0.005687,0.006829,0.003039,0.004267,-0.001884,-0.001228,0.004473,0.000952,0.003519,...,0.003174,0.007164,-0.002838,-0.006004,0.007055,0.001272,0.004244,0.005165,-0.000163,0.012917,-0.001646,-0.006065,-0.003731,-0.004595,-0.008753,0.008426,-0.009800,-0.005765,0.001794,0.006590,0.002274,0.006164,-0.003041,0.007426,0.007541,0.002908,-0.001028,-0.013441,-0.004015,0.005720,-0.005524,-0.001443,-0.003787,-0.007360,-0.007552,0.001964,0.002989,-0.001459,0.000290,CCOC1=C(C=CC(=C1)/C=C(\C#N)/C2=CC=C(C=C2)[N+](...
4,ImageData/uh4.png,0.008436,-0.005728,-0.009615,0.002174,0.001339,-0.000722,0.004063,-0.005061,0.002492,-0.007012,0.007083,0.000911,0.003359,0.008284,0.001846,0.003797,0.005212,-0.001835,-0.009057,-0.004958,-0.004126,-0.003551,0.002090,-0.006206,0.006468,-0.003893,0.009303,0.001653,0.000316,-0.001350,-0.003354,-0.005321,0.002547,0.000865,-0.002324,0.004561,-0.002550,-0.003157,0.005723,...,-0.006920,-0.000756,0.001109,0.000027,0.004413,-0.000395,-0.002

In [15]:
# import tensorflow as tf

# cifar = tf.keras.datasets.cifar100
# (x_train, y_train), (x_test, y_test) = cifar.load_data()
# model = tf.keras.applications.ResNet50(
#     include_top=True,
#     weights=None,
#     input_shape=(32, 32, 3),
#     classes=100,)

# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
# model.fit(x_train, y_train, epochs=5, batch_size=64)

In [13]:
#Split training and test data
# keras.preprocessing.image.load_img()
TrainX = np.asarray([keras.utils.img_to_array(keras.preprocessing.image.load_img(path=data['Images'][i], color_mode='grayscale').resize((150, 150))) for i in range(0, 1000)], dtype=np.float16)
TrainY = np.array([_getEmbedding(i, data.drop(['Images', 'SMILES'], axis=1)) for i in range(0, 1000)])#, dtype=np.float16)
TrainY_sm = data['SMILES'].iloc[0:1000].to_numpy()

TestX = np.asarray([keras.utils.img_to_array(keras.preprocessing.image.load_img(path=data['Images'][i], color_mode='grayscale').resize((150, 150))) for i in range(1000, 1249)], dtype=np.float16)
TestY = np.asarray([_getEmbedding(i, data.drop(['Images', 'SMILES'], axis=1)) for i in range(1000, 1249)])#, dtype=np.float16)
TestY_sm = data['SMILES'].iloc[10000:1249].to_numpy()

AllEmbeddings = np.concatenate((TrainY, TestY))

In [14]:
TrainX.shape, TrainY.shape, TrainY_sm.shape#, TrainY[0]

((1000, 150, 150, 1), (1000, 128), (1000,))

In [93]:
# TrainY_sm[1249]

'C1=CC=C(C=C1)CCN(C2=CC=C(C=C2)NC(=S)NC(=O)C3=CC=CS3)C(=S)NC(=O)C4=CC=CS4'

In [15]:
TrainY[0]

array([ 8.9177275e-03,  7.1854717e-03,  3.8428665e-03, -4.0253940e-03,
       -7.1591362e-03, -4.3388084e-03, -2.2244342e-03,  4.5708410e-03,
       -6.6944393e-03, -7.9990910e-03, -4.2905040e-03,  1.1734873e-02,
       -6.5311664e-03,  1.9662790e-03,  6.8848200e-03,  6.0130400e-03,
        6.6165370e-04,  5.5164765e-03, -6.4791483e-03,  4.8608356e-03,
       -1.8425450e-04, -5.2225525e-03, -1.9734392e-03, -7.1963600e-03,
       -7.5456296e-04, -8.2247204e-04,  9.5439370e-03, -5.3567146e-03,
       -8.1392920e-03, -3.4479846e-04, -7.4726716e-03, -6.9147090e-03,
       -1.3519467e-03, -4.7649564e-03, -6.3540163e-03,  9.1820860e-04,
       -1.7171575e-03,  3.8439300e-03,  5.0288877e-03, -7.0183566e-03,
       -6.3349665e-03, -9.6803333e-04, -3.5540640e-03,  3.4882694e-03,
        7.1226470e-03,  4.0810510e-03, -5.2643553e-03, -7.9339750e-04,
        4.9102370e-04, -7.8875620e-03,  1.0330998e-02,  7.8057590e-03,
        3.1532740e-03,  1.5166359e-03,  5.3257262e-03,  7.5360010e-03,
      

In [238]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
# model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))

model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(128))

model.summary()

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

'''training model USING GENERATOR 
history = model.fit(generator(Data['Images'], Data.drop(['Images'], axis=1), 32),
 validation_data = (TestX, TestY), steps_per_epoch = 1000 // 32,
 epochs = 10)


#Train model
history = model.fit(TrainX, TrainY,
 validation_data = (TestX, TestY), 
 epochs = 10)
'''
history = model.fit(TrainX, TrainY,
 validation_data = (TestX, TestY), 
 epochs = 3)

# Running samples
#sample_chemical_embed = AllEmbeddings[50]
sample_chemical_diagram = TrainX[50]
#model_prediction = model.predict([sample_chemical_diagram])
model_prediction = model(np.reshape(sample_chemical_diagram, (1, 150, 150)))
closest_embed = _get_closest_embedding(vector=model_prediction[0], embeds=AllEmbeddings)[0]

# TODO retrieve corresponding iupac, compare w/ actual iupac
 
#def loadAndTrainLSTM()

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 147968)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    18,940,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,032,704 (72.60 MB)

 Trainable params: 19,032,704 (72.60 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
 9/32 ━━━━━━━━━━━━━━━━━━━━ 13s 570ms/step - accuracy: 0.0026 - loss: 147831.0625

KeyboardInterrupt: 

In [39]:
history.history['accuracy']
!pip show keras


/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Name: keras
Version: 3.3.2
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /opt/homebrew/lib/python3.11/site-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, rich
Required-by: tensorflow-macos


In [ ]:
"""Vectorization"""

# vectorize_layer = TextVectorization(
#         max_tokens=vocab_size,
#         output_mode="int",
#         standardize=custom_standardization,
#         output_sequence_length=max_seq,
#     )
#     vectorize_layer.adapt(texts)

In [239]:
model.layers

[<Conv2D name=conv2d, built=True>,
 <MaxPooling2D name=max_pooling2d, built=True>,
 <Conv2D name=conv2d_1, built=True>,
 <MaxPooling2D name=max_pooling2d_1, built=True>,
 <Conv2D name=conv2d_2, built=True>,
 <Flatten name=flatten, built=True>,
 <Dense name=dense, built=True>]

In [ ]:
"""Functional model"""
# from keras.layers import MultiHeadAttention, Dense, BatchNormalization, Input, 
# timestep_dimension = self.num_exercises*self.num_exercise_features

# # Timestep layers
# input_timesteps = Input(shape=(self.num_steps, timestep_dimension))
# lstm_1 = LSTM(units=125, activation="tanh", return_sequences=False)(input_timesteps)
# batch_norm_1 = BatchNormalization()(lstm_1)
# dense_1 = Dense(units=timestep_dimension/2)(batch_norm_1)

# embeds = np.array(self.embeddings)
# downsampled_embeds = np.array(Dense(units=timestep_dimension/2)(embeds))
# def concate_in_embeddings(downsampled_timesteps):
#     batch_size = K.shape(downsampled_timesteps)[0]
#     tiled_embeddings = K.tile(np.array([downsampled_embeds]), (batch_size, 1, 1))
#     return tiled_embeddings
    
# embeddings_processed = Lambda(concate_in_embeddings)(dense_1)
# downsampled_embeddings = embeddings_processed # Consider adding more layers in between


# mat_multed_timesteps = embeddings_multiplier([downsampled_embeddings, dense_1])
# batch_norm_2 = BatchNormalization()(mat_multed_timesteps)
# post_mat_layer_1 = Dense(units=timestep_dimension*2)(batch_norm_2)

# # Probability layer
# prob_layer = tfp.layers.DistributionLambda(
#     lambda t: tfd.Normal(loc=t[..., :timestep_dimension], 
#                         scale=0.01*tf.math.softplus(t[..., timestep_dimension:])))(post_mat_layer_1)#(tf.Variable(post_mat_layer_1))
# output = prob_layer # maybe add more layers

# model = Model(inputs=input_timesteps, outputs=output)
# #model.compile(optimizer="RMSprop", loss="mse", metrics=None)
# model.compile(optimizer="RMSprop", loss=neg_log_lik)

# self.model = model


In [16]:
tf.keras.preprocessing.text.Tokenizer()
# print(data)
smiles = data['SMILES']
smiles

0        C[NH+](CC1=C(C=CC=C1Cl)F)CN2C(=O)CC3(C2=O)CCCCC3
1       CC1=C(C=C(C=C1)C(=O)NC2=C(C=C(C=C2)I)C3=NC4=CC...
2       COC1=CC=C(C=C1)CCN2C(=O)CC(SC2=NC3=CC=C(C=C3)O...
3       CCOC1=C(C=CC(=C1)/C=C(\C#N)/C2=CC=C(C=C2)[N+](...
4       CC1=C(C(=O)N(N1C)C2=CC=CC=C2)CN(CCN3CCCC3)C(=O...
                              ...                        
1245    C1=CC=C(C(=C1)NC(=O)C2=C(C(=CC(=C2)Cl)[N+](=O)...
1246     CC1=CC2=C(C=C1O)C(=C(N2CC3=CC(=CC=C3)Cl)C)C(=O)N
1247       C[C@H]1C(=O)NC2=C(O1)C=CC(=C2)C(=O)[C@@H](C)Cl
1248       C1C2=C(C=C(C=C2)NC(=O)NC3=CC=CC=C3Br)NS1(=O)=O
1249    C1=CC=C(C=C1)CCN(C2=CC=C(C=C2)NC(=S)NC(=O)C3=C...
Name: SMILES, Length: 1250, dtype: object

In [4]:
# !pip install --pre deepchem
# import deepchem
# deepchem.molnet.TransformerGenerator.create_transformer
# import deepchem as dc
# # deepchem.trans.Transformer.
# import transformers
smiles_tokenizer = dc.feat.SmilesTokenizer('vocab.txt')#deepchem/feat/tests/data/vocab.txt')

In [8]:
smiles_tokenizer('====')['input_ids']

[12, 22, 22, 22, 22, 13]

In [25]:
list(tf.data.Dataset.from_tensor_slices((TrainX, TrainY)))[0][0].shape, \
list(tf.data.Dataset.from_tensor_slices((TrainX, TrainY)))[0][1].shape,

(TensorShape([150, 150, 1]), TensorShape([128]))

In [47]:
emb_ds=tf.data.Dataset.from_tensor_slices((TrainX, TrainY)).padded_batch(batch_size=64)

In [49]:
# history = model.fit(emb_ds, validation_data = (TestX, TestY), epochs=20)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 527ms/step - accuracy: 0.0171 - loss: 4.0014e-05 - val_accuracy: 0.0201 - val_loss: 3.9936e-05
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 509ms/step - accuracy: 0.0171 - loss: 3.9964e-05 - val_accuracy: 0.0201 - val_loss: 3.9885e-05
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 502ms/step - accuracy: 0.0171 - loss: 3.9914e-05 - val_accuracy: 0.0201 - val_loss: 3.9834e-05
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 503ms/step - accuracy: 0.0171 - loss: 3.9863e-05 - val_accuracy: 0.0201 - val_loss: 3.9782e-05
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 512ms/step - accuracy: 0.0171 - loss: 3.9811e-05 - val_accuracy: 0.0201 - val_loss: 3.9730e-05
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 516ms/step - accuracy: 0.0171 - loss: 3.9759e-05 - val_accuracy: 0.0201 - val_loss: 3.9677e-05
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 514ms/step - accuracy: 0.0171 - loss: 3.9706e-05 - val_accuracy: 0.0201 - val_loss: 3.9624e-05
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 536ms/s

In [35]:
# smiles_tokenizer.vocab_size

591

In [45]:
one_hot_layer = tf.keras.layers.Lambda(lambda x: tf.one_hot(x, smiles_tokenizer.vocab_size))
# one_hot_layer(np.array([[34, 66]], dtype=np.int32))

tf.one_hot(np.array([0, 34, 66]), smiles_tokenizer.vocab_size)

<tf.Tensor: shape=(3, 591), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [1]:
from tensorflow.keras.layers import Dense, LSTM, Dropout, Reshape, concatenate, Input, Lambda, Flatten, Activation, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from keras_nlp.layers import TransformerEncoder, TransformerDecoder
class TfProbabilityModel:
    pass
class ProbabilityEmbeddingsModel(TfProbabilityModel):
    def __init__(self, num_exercises=700, n_steps=10, n_features_per_ex=2, n_lstm_units=125, dropout_enabled=True, hyper_params=None, pre_trained_model=None, dir=None, metadata=None, embeds=None):
        self.embeddings_model = sentence_embedding()

        #if not (isinstance(embeds, np.ndarray) and not isinstance(embeds, list)) or embeds == None or len(embeds) == 0:
        #    print("WARNING: No Embeddings Found; Model will not compile. Recompile model with embeddings sent")
        #    return

        if metadata != None:
            # Load from metadata
            self.model_metadata = metadata
            self.num_exercises = metadata['num_exercises']
            self.num_steps = metadata['num_steps']
            self.num_exercise_features = metadata['num_exercise_features']
            self.dropout_enabled = metadata['dropout_enabled']
            self.num_lstm_units = metadata['num_lstm_units']
            self.hyper_parameters = metadata['hyper_parameters']
            self.custom_objects = None
            self.embeddings = metadata['embeddings']
            if dir != None:
                self.model = load_model(dir)
                return
                
        if embeds is None:
            print("WARNING: No Embeddings Found; Model will not compile. Recompile model with embeddings sent")
            return
        # Initialize/get embeddings
        self.embeddings = embeds

        super().__init__(num_exercises, n_steps, n_features_per_ex, n_lstm_units, dropout_enabled, hyper_params, pre_trained_model)
        self.set_model_metadata(model_type='ProbabilityEmbeddingsModel')

    def set_model_metadata(self, model_type):
        self.model_metadata['embeddings'] = self.embeddings.tolist()
        super().set_model_metadata(model_type)

    def _compile(self):
        #if self.embeddings == None or len(self.embeddings) == 0:
        #    print("WARNING: No Embeddings Found; Model will not compile. Recompile model with embeddings sent")
        #    return

        timestep_dimension = self.num_exercises*self.num_exercise_features

        # Timestep layers
        input_timesteps = Input(shape=(self.num_steps, timestep_dimension))
        lstm_1 = LSTM(units=125, activation="tanh", return_sequences=False)(input_timesteps)
        batch_norm_1 = BatchNormalization()(lstm_1)
        dense_1 = Dense(units=timestep_dimension/2)(batch_norm_1)

        embeds = np.array(self.embeddings)
        downsampled_embeds = np.array(Dense(units=timestep_dimension/2)(embeds))
        def concate_in_embeddings(downsampled_timesteps):
            batch_size = K.shape(downsampled_timesteps)[0]
            tiled_embeddings = K.tile(np.array([downsampled_embeds]), (batch_size, 1, 1))
            return tiled_embeddings
            
        embeddings_processed = Lambda(concate_in_embeddings)(dense_1)
        downsampled_embeddings = embeddings_processed # Consider adding more layers in between

        class EmbeddingsMatMultLayer(keras.layers.Layer):
            def __init__(self):
                super().__init__()
                
            def call(self, inputs):
                x, y = inputs
                return K.batch_dot(x, y)
            
        embeddings_multiplier = EmbeddingsMatMultLayer()
        mat_multed_timesteps = embeddings_multiplier([downsampled_embeddings, dense_1])
        batch_norm_2 = BatchNormalization()(mat_multed_timesteps)
        post_mat_layer_1 = Dense(units=timestep_dimension*2)(batch_norm_2)

        # Probability layer
        prob_layer = tfp.layers.DistributionLambda(
            lambda t: tfd.Normal(loc=t[..., :timestep_dimension], 
                                scale=0.01*tf.math.softplus(t[..., timestep_dimension:])))(post_mat_layer_1)#(tf.Variable(post_mat_layer_1))
        output = prob_layer # maybe add more layers

        model = Model(inputs=input_timesteps, outputs=output)
        #model.compile(optimizer="RMSprop", loss="mse", metrics=None)
        model.compile(optimizer="RMSprop", loss=neg_log_lik)

        self.model = model
    
    def train(self, x, y, epochs=50, batch_size=32, use_cache=True): # TODO
        return super().train(x, y, epochs, batch_size, use_cache)

In [77]:
# model.save('encoder.keras')
model.layers[0:4]
# Sequential(model.layers[0:4])

[<Conv2D name=conv2d, built=True>,
 <MaxPooling2D name=max_pooling2d, built=True>,
 <Conv2D name=conv2d_1, built=True>,
 <MaxPooling2D name=max_pooling2d_1, built=True>]

In [71]:
# smiles_tokenizer = dc.feat.SmilesTokenizer('vocab.txt')#deepchem/feat/tests/data/vocab.txt')
tv=None
tv = keras.layers.TextVectorization(vocabulary='vocafb.txt', output_mode='int', standardize=None, split='character')


In [72]:
tv.get_vocabulary()#include_special_tokens=False)

['',
 '[UNK]',
 '[SOS]',
 '[EOS]',
 'c',
 'C',
 '(',
 ')',
 'O',
 '1',
 '2',
 '=',
 'N',
 '.',
 'n',
 '3',
 'F',
 'Cl',
 '>>',
 '~',
 '-',
 '4',
 '[C@H]',
 'S',
 '[C@@H]',
 '[O-]',
 'Br',
 '#',
 '/',
 '[nH]',
 '[N+]',
 's',
 '5',
 'o',
 'P',
 '[Na+]',
 '[Si]',
 'I',
 '[Na]',
 '[Pd]',
 '[K+]',
 '[K]',
 '[P]',
 'B',
 '[C@]',
 '[C@@]',
 '[Cl-]',
 '6',
 '[OH-]',
 '\\',
 '[N-]',
 '[Li]',
 '[H]',
 '[2H]',
 '[NH4+]',
 '[c-]',
 '[P-]',
 '[Cs+]',
 '[Li+]',
 '[Cs]',
 '[NaH]',
 '[H-]',
 '[O+]',
 '[BH4-]',
 '[Cu]',
 '7',
 '[Mg]',
 '[Fe+2]',
 '[n+]',
 '[Sn]',
 '[BH-]',
 '[Pd+2]',
 '[CH]',
 '[I-]',
 '[Br-]',
 '[C-]',
 '[Zn]',
 '[B-]',
 '[F-]',
 '[Al]',
 '[P+]',
 '[BH3-]',
 '[Fe]',
 '[C]',
 '[AlH4]',
 '[Ni]',
 '[SiH]',
 '8',
 '[Cu+2]',
 '[Mn]',
 '[AlH]',
 '[nH+]',
 '[AlH4-]',
 '[O-2]',
 '[Cr]',
 '[Mg+2]',
 '[NH3+]',
 '[S@]',
 '[Pt]',
 '[Al+3]',
 '[S@@]',
 '[S-]',
 '[Ti]',
 '[Zn+2]',
 '[PH]',
 '[NH2+]',
 '[Ru]',
 '[Ag+]',
 '[S+]',
 '[I+3]',
 '[NH+]',
 '[Ca+2]',
 '[Ag]',
 '9',
 '[Os]',
 '[Se]',
 '[SiH2

In [121]:
# re.findall('([0-2])|(1)', '-1-1-1')
list(re.finditer('(^[nH])|([AlH4-])|([Cr])|([Mg+2])', '[nH]+AlH4-'))

[<re.Match object; span=(2, 3), match='H'>,
 <re.Match object; span=(4, 5), match='+'>,
 <re.Match object; span=(5, 6), match='A'>,
 <re.Match object; span=(6, 7), match='l'>,
 <re.Match object; span=(7, 8), match='H'>,
 <re.Match object; span=(8, 9), match='4'>,
 <re.Match object; span=(9, 10), match='-'>]

In [283]:
tv("[Ho]")
# arr = np.sort(np.loadtxt(fname='vocafb.txt', dtype=str)[::-1])
arr = sorted(np.loadtxt(fname='vocafb.txt', dtype=str), key=len, reverse=True)

# pattern = '(?u-' + ')|(?u-'.join(arr) + ')'

# arr = np.vectorize(re.escape)(arr)

# pattern = re.escape('(' + ')|('.join(arr) + ')')
# pattern = '(' + ')|('.join(arr) + ')'

# pattern = '|'.join(f'((<=.*){re.escape(v)})' for i, v in enumerate(arr)) + '|(?P<unk>.+)'
pattern = '|'.join(f'({re.escape(v)})' for i, v in enumerate(arr)) + '|(?P<unk>.+?)'

# pattern = '|'.join(f'({re.escape(v)})' for i, v in enumerate(arr)) + '|(?P<unk>.+?)'

pattern = '|'.join(f'({re.escape(v)})' for i, v in enumerate(arr)) + '|(?P<unk>.+?)'

pattern#[550:600]
len(arr)
arr
# arr

['[13C@@H]',
 '[67Ga+3]',
 '[AlH4-]',
 '[AlH2-]',
 '[13CH3]',
 '[13CH2]',
 '[11CH3]',
 '[15NH2]',
 '[Si@@H]',
 '[123I-]',
 '[14CH3]',
 '[ClH2+]',
 '[13C@H]',
 '[N@@H+]',
 '[AsH4+]',
 '[14C@H]',
 '[211At]',
 '[NH3+2]',
 '[AlH2+]',
 '[229Th]',
 '[PtH+2]',
 '[14CH2]',
 '[SiH4+]',
 '[18OH2]',
 '[118Sn]',
 '[153Sm]',
 '[24NaH]',
 '[124I-]',
 '[CuH2-]',
 '[15OH2]',
 '[C@@H]',
 '[NH4+]',
 '[BH4-]',
 '[Fe+2]',
 '[Pd+2]',
 '[BH3-]',
 '[AlH4]',
 '[Cu+2]',
 '[Mg+2]',
 '[NH3+]',
 '[Al+3]',
 '[Zn+2]',
 '[NH2+]',
 '[Ca+2]',
 '[SiH2]',
 '[Ti+4]',
 '[Cl+3]',
 '[Pt+2]',
 '[SiH3]',
 '[Fe+3]',
 '[CH2+]',
 '[CH2-]',
 '[Ce+4]',
 '[Pt+4]',
 '[Sn+2]',
 '[Ba+2]',
 '[Fe-3]',
 '[Pb+2]',
 '[Os-2]',
 '[Zr+4]',
 '[Ni+2]',
 '[Co+2]',
 '[Hg+2]',
 '[Yb+3]',
 '[Zr+2]',
 '[Mn+2]',
 '[Ce+3]',
 '[OH2+]',
 '[Ti+3]',
 '[Rh+2]',
 '[Si@H]',
 '[Mn+4]',
 '[NH2-]',
 '[Pb+4]',
 '[Cr+3]',
 '[Sc+3]',
 '[Rh-3]',
 '[PH4+]',
 '[SiH4]',
 '[CH3-]',
 '[S@@+]',
 '[Rh+3]',
 '[Bi+3]',
 '[Br+2]',
 '[La+3]',
 '[Pt-2]',
 '[PH3+]',
 '[Si+4]',


In [284]:
# from numpy import getarr
import re
# arr = np.loadtxt(fname='vocafb.txt', dtype=str)
# arr
found_iter = re.finditer(pattern=pattern, string='[SOS]C1=C(C)C=CC=C1[nH][O-2]iiiiC[EOS]')#, flags=re.U)
# list(found_iter)[0].groupdict()
# found_iter = re.findall(pattern=pattern, string='.n')#, flags=re.U)
# found_iter
# list(found_iter)
found_iter = list(found_iter)
found_iter

[<re.Match object; span=(0, 5), match='[SOS]'>,
 <re.Match object; span=(5, 6), match='C'>,
 <re.Match object; span=(6, 7), match='1'>,
 <re.Match object; span=(7, 8), match='='>,
 <re.Match object; span=(8, 9), match='C'>,
 <re.Match object; span=(9, 10), match='('>,
 <re.Match object; span=(10, 11), match='C'>,
 <re.Match object; span=(11, 12), match=')'>,
 <re.Match object; span=(12, 13), match='C'>,
 <re.Match object; span=(13, 14), match='='>,
 <re.Match object; span=(14, 15), match='C'>,
 <re.Match object; span=(15, 16), match='C'>,
 <re.Match object; span=(16, 17), match='='>,
 <re.Match object; span=(17, 18), match='C'>,
 <re.Match object; span=(18, 19), match='1'>,
 <re.Match object; span=(19, 23), match='[nH]'>,
 <re.Match object; span=(23, 28), match='[O-2]'>,
 <re.Match object; span=(28, 29), match='i'>,
 <re.Match object; span=(29, 30), match='i'>,
 <re.Match object; span=(30, 31), match='i'>,
 <re.Match object; span=(31, 32), match='i'>,
 <re.Match object; span=(32, 33), 

In [285]:
for match in found_iter:
    print(match.lastindex)

230
547
551
553
547
548
547
549
547
553
547
547
553
547
551
391
245
578
578
578
578
547
231


In [290]:

def tokenize(smiles):
    found_iter = re.finditer(pattern=pattern, string=smiles)
    return np.array([found.lastindex for found in found_iter], dtype=np.int32)


tokenize('C=Cl')
tokenize(';')

array([578], dtype=int32)

In [292]:
# tokenized = tf.keras.layers.Lambda(tokenize)
# tokenized(np.array(['C==C']))

In [296]:
print(TrainY_sm[0])
f_iter = re.finditer(pattern, TrainY_sm[0])#[17]
# list(f_iter)

for j in f_iter:
    if j.lastindex == 578:
        print(j)

C[NH+](CC1=C(C=CC=C1Cl)F)CN2C(=O)CC3(C2=O)CCCCC3


In [294]:
TrainY_sm_tok = [tokenize('[SOS]' + smile + '[EOS]') for smile in TrainY_sm]#np.vectorize(tokenize)(TrainY_sm)
TrainY_sm_tok

[array([230, 547, 249, 548, 547, 547, 551, 553, 547, 548, 547, 553, 547,
        547, 553, 547, 551, 543, 549, 558, 549, 547, 554, 552, 547, 548,
        553, 550, 549, 547, 547, 557, 548, 547, 552, 553, 550, 549, 547,
        547, 547, 547, 547, 557, 231], dtype=int32),
 array([230, 547, 547, 551, 553, 547, 548, 547, 553, 547, 548, 547, 553,
        547, 551, 549, 547, 548, 553, 550, 549, 554, 547, 552, 553, 547,
        548, 547, 553, 547, 548, 547, 553, 547, 552, 549, 568, 549, 547,
        557, 553, 554, 547, 561, 553, 547, 547, 553, 547, 547, 553, 547,
        561, 562, 557, 549, 543, 231], dtype=int32),
 array([230, 547, 550, 547, 551, 553, 547, 547, 553, 547, 548, 547, 553,
        547, 551, 549, 547, 547, 554, 552, 547, 548, 553, 550, 549, 547,
        547, 548, 562, 547, 552, 553, 554, 547, 557, 553, 547, 547, 553,
        547, 548, 547, 553, 547, 557, 549, 550, 547, 549, 547, 548, 553,
        550, 549, 554, 547, 561, 553, 547, 547, 548, 553, 547, 547, 553,
        547, 561, 

In [302]:
# tokenizer = tfds.features.text.Tokenizer()


# # Create an encoder instance given our vocabulary set.
# encoder = tfds.features.text.TokenTextEncoder(arr)

NameError: name 'tfds' is not defined

In [309]:
# import tfds
ragged_TrainY_tok = tf.ragged.constant(TrainY_sm_tok)
ragged_TrainY_tok = ragged_TrainY_tok.to_tensor()
ragged_sm_dataset = tf.data.Dataset.from_tensor_slices(ragged_TrainY_tok).padded_batch(
    batch_size=32,
    padding_values=0
)
list(ragged_sm_dataset)

[<tf.Tensor: shape=(32, 81), dtype=int32, numpy=
 array([[230, 547, 249, ...,   0,   0,   0],
        [230, 547, 547, ...,   0,   0,   0],
        [230, 547, 550, ...,   0,   0,   0],
        ...,
        [230, 547, 551, ...,   0,   0,   0],
        [230, 547, 547, ...,   0,   0,   0],
        [230, 547, 547, ...,   0,   0,   0]], dtype=int32)>,
 <tf.Tensor: shape=(32, 81), dtype=int32, numpy=
 array([[230, 547, 551, ...,   0,   0,   0],
        [230, 547, 547, ...,   0,   0,   0],
        [230, 547, 551, ...,   0,   0,   0],
        ...,
        [230, 547, 547, ...,   0,   0,   0],
        [230, 547, 547, ...,   0,   0,   0],
        [230, 547,  31, ...,   0,   0,   0]], dtype=int32)>,
 <tf.Tensor: shape=(32, 81), dtype=int32, numpy=
 array([[230, 547, 547, ...,   0,   0,   0],
        [230, 547, 551, ...,   0,   0,   0],
        [230, 547, 550, ...,   0,   0,   0],
        ...,
        [230, 547, 547, ...,   0,   0,   0],
        [230, 547, 547, ...,   0,   0,   0],
        [230, 547

In [312]:
ragged_TrainY_tok[50]

<tf.Tensor: shape=(81,), dtype=int32, numpy=
array([230, 562, 578, 568, 578, 578, 562, 231,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)>

In [301]:
counter=-1
def get_next():
    counter+=1
    yield TrainY_sm_tok[counter]


tf.data.Dataset.from_generator(get_next).padded_batch(batch_size=64)

TypeError: To specify the output signature you need to provide either the `output_signature` argument or the `output_types` argument.

In [246]:
vocab = arr

img_input = Input(shape=(150, 150, 1))
encoded = Sequential(model.layers[0:5])(img_input)
encoded = Reshape((encoded.shape[-3]*encoded.shape[-2], encoded.shape[-1]))(encoded)
# encoded = Flatten(data_format='channels_last')(encoded) # -> 32x32 , 128
# encoder = 

smiles_input = Input(shape=(None,))

# tokenized = tf.keras.layers.Lambda(tokenize)
# one_hot_layer = tf.keras.layers.Lambda(lambda x: tf.one_hot(x, smiles_tokenizer.vocab_size))
# one_hotted = tf.keras.layers.Lambda(lambda x: tf.one_hot(x, smiles_tokenizer.vocab_size))(smiles_input)
one_hotted = tf.keras.layers.CategoryEncoding(num_tokens=len(vocab)+1, output_mode="one_hot")(smiles_input)
embedded_smiles = tf.keras.layers.Embedding(input_dim=len(vocab), output_dim=128)(one_hotted)#_layer)

decoded = TransformerDecoder(
    intermediate_dim=128,
    num_heads=2,
    dropout=0,
    activation="relu",
    layer_norm_epsilon=1e-05,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    normalize_first=False)(encoder_sequence=encoded, decoder_sequence=embedded_smiles)#one_hot_layer)

tr_model = Model(inputs=img_input, outputs=encoded)
tr_model.compile()


In [247]:
tr_model.predict(np.array([TrainX[0]])).shape#np.random((1, 150, 150, 1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step


(1, 1156, 128)

In [67]:
# TrainX[0].shape

(150, 150, 1)